<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

**Описание данных** 

Датафрейм содержит исторические данные о поведении клиентов и расторжении договоров с банком:

- RowNumber — индекс строки в данных,

- CustomerId — уникальный идентификатор клиента,

- Surname — фамилия,

- CreditScore — кредитный рейтинг,

- Geography — страна проживания,

- Gender — пол,

- Age — возраст,

- Tenure — сколько лет человек является клиентом банка,

- Balance — баланс на счёте,

- NumOfProducts — количество продуктов банка, используемых клиентом,

- HasCrCard — наличие кредитной карты,

- IsActiveMember — активность клиента,

- EstimatedSalary — предполагаемая зарплата.


Целевой признак

- Exited — факт ухода клиента.


## Подготовка данных

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from tqdm import tqdm

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/Churn.csv')

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data

In [ ]:
data.columns = map(str.lower, data.columns)

In [ ]:
data

In [ ]:
data.rename(columns = {'customerid': 'customer_id', 'creditscore': 'credit_score', 'numofproducts': 'num_of_products', 'hascrcard': 'cr_card', 'isactivemember': 'is_active_member', 'estimatedsalary': 'estimated_salary'}, inplace = True)

In [ ]:
data.columns

In [ ]:
data = data.sample(frac=1, random_state=12345).drop(['surname', 'rownumber', 'customer_id'], axis=1)

In [ ]:
data = pd.get_dummies(data, drop_first=True)

In [ ]:
data

In [ ]:
data.duplicated().sum()

In [ ]:
data.isna().sum()

In [ ]:
data['tenure'].value_counts()

In [ ]:
from random import randint
import math
def nan_random(v):
    if math.isnan(v):
        return randint(0, 10)
    else:
        return v

In [ ]:
data['tenure'] = data['tenure'].apply(nan_random)

In [ ]:
data['tenure'].isna().sum()

In [ ]:
data['tenure'].value_counts()

In [ ]:
features = data.drop(['exited'], axis=1)
target = data['exited']

In [ ]:
features

In [ ]:
features_train, features_df, target_train, target_df = train_test_split(
    features, target, test_size=0.4, random_state=12345, stratify=target)

In [ ]:
print(features_train.shape)

In [ ]:
features_test, features_valid, target_test, target_valid = train_test_split(
    features_df, target_df, test_size=0.5, random_state=12345, stratify=target_df)

In [ ]:
numeric = ['credit_score', 'age', 'balance', 'estimated_salary', 'num_of_products']

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])
features_train.head()

**Вывод:** 

данные содержат 10 тысяч строк в 14 столбцах. В ходе предобработки я удалила 2 столбца: 'Surname' содержащий фамилии и 'RowNumber' задвоивший порядковый номер строк. Пропуски обнаружены в столбце 'tenure' в количестве почти 10% от отщего числа строк. Я сначала попыталась восстановить их с помощью импутера, но отследила через value_counts, что импутер добавил среднее значение и "5" стало на 909 больше, за счет чего сильно изменился баланс. Поэтому я решила просто не учитывать этот столбец в фичах. Если удалить эти строки, то почти 10% данных полеряются и в других столбцах. 

Применила стандартизацию для уравнения значимости всех признаков.

## Исследование задачи

In [ ]:
class_frequency = target.value_counts(normalize=True)
print(class_frequency)
class_frequency.plot(kind='bar');

***Комментарий:*** 
соотношение классов 80:20: данные содержат 80% информации о действующих клиентах банка и 20% об ушедших.

## Борьба с дисбалансом

***Взвешивание классов***

In [ ]:
best_model_rfc_balance = None
best_result_rfc_balance = 0

for depth in tqdm(range(3, 50, 1)):
    
    for est in range(50, 250, 50):
        model = RandomForestClassifier(class_weight='balanced', random_state=12345, max_depth=depth, n_estimators=est, criterion='gini') 
        model.fit(features_train, target_train) 
        predictions = model.predict(features_valid) 
        result = f1_score(target_valid, predictions) 
        if result > best_result_rfc_balance:
            best_model_rfc_balance = model
            best_result_rfc_balance = result
print("f1 лучшей модели:", best_result_rfc_balance)
print("Наилучшая модель:", best_model_rfc_balance)

***Oversample***

In [ ]:
num = 3
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, num)

In [ ]:
class_frequency = target_upsampled.value_counts(normalize=True)
print(class_frequency)
class_frequency.plot(kind='bar');

In [ ]:
best_upsample_model = None
best_upsample_result = 0

for depth in tqdm(range(3, 50, 1)):
    
    for est in range(50, 250, 50):
        model = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=est, criterion='gini') 
        model.fit(features_upsampled, target_upsampled) 
        predictions = model.predict(features_valid) 
        result = f1_score(target_valid, predictions) 
        if result > best_upsample_result:
            best_upsample_model = model
            best_upsample_result = result
print("f1 лучшей модели:", best_upsample_result)
print("Наилучшая модель:", best_upsample_model)

In [ ]:
print(confusion_matrix(target_valid, (best_upsample_model.predict(features_valid))))

In [ ]:
model_upsample_lr = LogisticRegression(random_state=12345, solver='liblinear')
model_upsample_lr.fit(features_upsampled, target_upsampled)
predicted_valid = model_upsample_lr.predict(features_valid)
print("F1 после баланса:", f1_score(target_valid, predicted_valid))

**Вывод:** 
после работы над дисбалансом классов метрика f1 моделей возрасла. Я применила взвешивание классов и upsampling для модели логистической регресии и случайного леса. Лучший результат показала модель RandomForest upsampling, ее и оставим для тестирования.

## Тестирование модели

In [ ]:
test_predictions = best_upsample_model.predict(features_test)

In [ ]:
print("F1:", f1_score(target_test, test_predictions))

In [ ]:
print(confusion_matrix(target_test, (best_upsample_model.predict(features_test))))

***Комментарий***
модель справляется с положительными и отрицательными ответами, но в FN и FP все еще попадает много.

In [ ]:
fpr, tpr, thresholds = roc_curve(target_test, best_upsample_model.predict_proba(features_test)[:,1]) 

plt.figure()
plt.title('ROC-кривая')
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.ylim([0.0, 1.0])
plt.xlim([0.0, 1.0])

In [ ]:
auc_roc = roc_auc_score(target_test, best_upsample_model.predict_proba(features_test)[:, 1])
print(auc_roc)

**Вывод:**  

у модели достаточно высокий roc_auc: 0.85, она неплохо справляется с класификацией положительных и отрицательных ответов, f1 достигает 0.61.

In [ ]:
test_predictions_balance = best_model_rfc_balance.predict(features_test)
print("F1:", f1_score(target_test, test_predictions_balance))

In [ ]:
print(confusion_matrix(target_test, (best_model_rfc_balance.predict(features_test))))

In [ ]:
fpr, tpr, thresholds = roc_curve(target_test, best_model_rfc_balance.predict_proba(features_test)[:,1]) 

plt.figure()
plt.title('ROC-кривая')
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.ylim([0.0, 1.0])
plt.xlim([0.0, 1.0])

In [ ]:
auc_roc = roc_auc_score(target_test, best_model_rfc_balance.predict_proba(features_test)[:, 1])
print(auc_roc)

### Вывод:

Датафрейм содержит исторические данные - каждый объект в наборе данных это информация о поведении одного клиента. Целевой признак - факт ухода клиента, данные содержат признаки, основываясь на которых модель должна предположить расторгнет ли пользователь договор с банком. Среди признаков: кредитный рейтинг, страна проживания, пол и возраст, баланс на счёте, количество продуктов банка, используемых клиентом, наличие кредитной карты, активность клиента и предполагаемая зарплата. 


В датафрейме хранятся 10 тысяч строк в 14 столбцах, данные были разбиты на 3 выборки в соотношении 60:20:20 - обучающая, валидационная и тестовая.
 

- соотношение классов 80:20: 80% информации о действующих клиентах банка и 20% об ушедших. 


Без баланса классов модель LogisticRegression находит лучше всего TN и FN. Почти все положительные ответы попадают в FN. 

- модель LogisticRegression f1 за счет баланса повысилась с 0.3 до 0.48;
- после баланса модель стала стала лучше находить единицы, но начала путать нули: появилось очень много FP ответов, до баланса их было всего 55, а теперь 470. FN до баланса было 323, а после стало 137. TP до баланса было 88, а после 270;
- среднегармоническое полноты и точности модели RF после баланса возросло с 0.56 до 0.61.

После работы над дисбалансом классов метрика f1 моделей возрасла. Я применила взвешивание классов для модели логистической регресии, для модели случайного леса лучший результат показала техника upsampling. 

 
Для работы с тестовой выборкой были взяты модели с наилушим показателем f1 - RF после баланса. На тестовой выборке roc_auc: 0.85, f1 достигает 0.61


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: задача исследована
    - [x]  Исследован баланс классов
    - [x]  Изучены модели без учёта дисбаланса
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 3: учтён дисбаланс
    - [x]  Применено несколько способов борьбы с дисбалансом
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 4: проведено тестирование
- [x]  Удалось достичь *F1*-меры не менее 0.59
- [x]  Исследована метрика *AUC-ROC*